In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pickle
import pandas as pd
import re
import time

In [2]:
# loop with this url to pull all users
user_list_url = 'https://www.anime-planet.com/anime/death-note/watched?page=1'

# use this url to loop through and get all anime titles
animes_url = 'https://www.anime-planet.com/anime/all?page=1'

In [3]:
# Function to parse through the url
def getAndParseUrl(url):
    results = requests.get(url)
    time.sleep(1)
    soup = BeautifulSoup(results.text, 'html.parser')
    return soup

Reload users list

In [4]:
with open('users.pickle', 'rb') as infile:
    users=pickle.load(infile)

infile.close()

Reload animes list

In [5]:
with open('animes.pickle', 'rb') as infile:
    animes=pickle.load(infile)

infile.close()

## Get Each User's Rating for Each Anime

In [270]:
# create the final list of dictionaries
# each dictionary in the list is a different user's rating of an individual anime
full_list_finals = []

In [1]:
# getting every user's rating for each anime

# each anime's overview page
for i in tqdm(range(4707,len(animes))):
    genres=[]
    anime_overview = getAndParseUrl(['https://www.'+url for url in animes.values()][i])
    userstats_urls = ['https://www.' + url + '/watched' for url in animes.values()]
    userstats = getAndParseUrl(userstats_urls[i])
#     # gets the max number of pages in user stats
#     userstats_pgs = userstats.findAll('ul', attrs={'class' : 'nav'})
#     max_us_pgs = int(userstats_pgs[0].findAll('a')[-2].text)
    
    # each anime's first 4 genres
    ag = anime_overview.select('li a[data-tooltip-entry]')
    try:
        for j in range(4):
            genres.append(ag[j].text.strip('\n'))
    except:
        try:
            for g in range(2):
                genres.append(ag[g].text.strip('\n'))
        except:
            try:
                for e in range(1):
                    genres.append(ag[e].text.strip('\n'))
            except:
                genres.append('None')
        # go through each user stat page per anime
    for u in range(1,6):
        user_desc = getAndParseUrl(userstats_urls[i]+f'?page={u}')

        # each user and the rating
        for k in range(0,len(user_desc.select('td a[href]'))):
            anime = {}
            # each anime title
            anime['title'] = anime_overview.select('h1')[0].text

            # each genre
            anime['genres'] = genres

            user = user_desc.select('td a[href]')[k].text
            rating = user_desc.findAll('div', {'class': 'starrating'})
            ratings = rating[k].text
            if user in users:
                anime['user'] = user
                anime['ratings'] = ratings
                rest_of_animes.append(anime)
            else: 
                continue
    print(full_list_finals)
    with open("full_list_finals.pickle", "wb") as rest_file:
        pickle.dump(full_list_finals, rest_file)


In [2]:
full_list_finals

In [274]:
with open('full_list_final.pickle', 'wb') as outfile:
    pickle.dump(full_list_final, outfile)

outfile.close()